### Newton Method

We will be using Newton method to find the nearest real root of a polynomial function

$$f(z)=a_nz+a_{n-1}z+\dots+a_1z+a_0$$

A general Newton method for finding roots of a function is described with formula

$$
z_{n+1}=z_n - \frac{f(z_n)}{f^\prime (z_n)}
$$

We represent a polynomial $f(z)$ as an array of its coefficients:

$$P=[a_0,a_1,\dots a_n]$$

In each step, we will calculate $f(z)$ and $f^\prime (z)$ using the Horner method reduced to last two terms. So, if we have the transformed function:

$$f(z)=a_0 + z(a_1 + z(a_2 + \dots + z(a_{n-1} + z\cdot a_n) \dots))$$

In the Horner method, for given $z_0$, we calculated a whole array ($b_{n-1}=a_n$, then $b_{k-1} = (a_k + z\cdot b_k)$ to finally get the last element $b_0$) which gave as a form:

$$f(z)=(z-z_0)q(z) + f(z_0)$$

Where $b_0, b_1,\dots,b_{n-1}$ are coefficients of $q(z)$. Now we will iteratively use the Horner method for $q$ to get $n$ vectors $b$ to finally get a form

$$
f(z)=b^{(n)}_0(z-z_0)^n + b^{(n-1)}_0(z-z_0)^{n-1} + \dots + b^{(1)}_0(z-z0) + b^{(0)}_0\\
f(z)=c_0 + c_1(z-z_0) + \dots + c_n(z-z_0)^n
$$

where $b^{(i)}$ is the result of the $i$-th iteration of Horner method. This by definition is a Taylor expansion of $f$ near some $z_0$. That means that we can take $f(z_0) = c_0=b^{(n)}_0$ which is the last element of the last iteration and $f^\prime (z_0)=c_1=b^{(n-1)}_0$ which is the last element of the last but one iteration. Each $c_i$ is the last element of the result of $i$-th iteration of the horner method ($b_0^{(i)}$)

As we can see, only the last element of $b$ is used in the expression. Let's call them $\alpha=b^{(n)}_0$ and $\beta=b^{(n-1)}_0$, in our new reduced Horner method we initialize:
$$
\alpha = a_n\\
\beta = 0
$$
then for given $z_0$ we iteratively overwrite $\alpha$ and $\beta$ for $k=n-1,n-2,\dots,1$.
$$
\beta^\ast := \alpha + z_0\beta \\
\alpha^\ast := a_k + z_0\alpha
$$
Virtually it is an unrolled version of the Horner method. The final result is $f(z_0) =\alpha$ and $f^\prime (z_0)=\beta$ which is all we need to perform one step of the Newton method.

$$z_{n+1}=z_n - \frac{\alpha}{\beta}$$

Note that the assignment of $\alpha$ and $\beta$ ends the first iteration of the full Horner method.

$$
\alpha = b_{n-1} = a_n\\
\beta^\ast = b_{n-2} = a_{n-1} + z_0 b_{n-1} = a_{n-1} + z_0 \cdot a_n = a_{n-1} + z_0\alpha^{(0)}
$$
In the next step of the full Horner method an input vector would be
$$[b_0,b_1,\dots ,b_{n-3},\beta]$$
In our case, we are interested only in $\alpha=b_{n-1}$ and $\beta=b_{n-2}$, so we don't store the $b$ array at all.



In [ ]:
%reset

In [27]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import itertools

P = [-2, -5, 7, -4, 1]
# P = [-6, 11, -6, 1]

def f(p, z):
    return sum([ a * z**k for k, a in enumerate(p)])

def horner_division(p, z):
    y=p[len(p)-1]
    b = [y]
    for a in np.flip(p[0:len(p)-1]):
        term = a +  y * z
        y = term
        b.append(term)
    return np.flip(b)

def horner(p, z):
    alpha=p[len(p)-1]
    beta = 0
    for a in np.flip(p[0:len(p)-1]):
        beta = alpha + z * beta
        alpha = a + z * alpha

    return alpha, beta

precision = 6
def find_root_taylor(p, z):
    z0 = z
    j = 0
    err = 1
    eps = 10 ** -precision
    steps = [z0]
    while j<50 and err > eps:
        alpha, beta = horner(p, z0)
        z1 = z0 - alpha/beta
        err = abs(z1 - z0)
        z0 = z1
        steps.append(z0)
        j = j+1
    print(f"root of {p} near " + "{:.7f}".format(z) + ": " + "{:.7f}".format(z0) + f" (steps: {j})")
    return round(z0, precision-1), steps

def find_roots_taylor(p, z):
    roots = []
    b = p
    for k in range(0, len(p) - 1):
        z0, _ = find_root_taylor(b, z)
        roots.append(z0)
        b = horner_division(b, z0)
        b = b[1:len(b)]
    print(f"All roots: {roots}.")
    return roots

# Z0 = np.linspace(-2, 2, 40)


%run ../print_utils.py
def rho(p):
    a_n = p[len(p)-1]
    max = np.max(p[0:len(p)-2])
    return 1 + abs(a_n**-1)*max

find_roots_taylor(P, 0.15)
numpyroots = np.roots(np.flip(P))
print(f'Numpy roots: {numpyroots}')

printpol(P)

X = np.linspace(-5, 5, 2000)
Y1 = [f(P, x) for x in X]


rho_val = rho(P)
invrho = 0
if rho_val != 0:
    invrho = 1.0/rho_val

printmd(r"\rho=" + str(rho_val))
printmd(r"z_0\in[" + str(invrho) + "," + str(rho_val) + "]")

rootRange = [ invrho, rho_val ]


z0, steps = find_root_taylor(P, -6)
def window(seq, n=2):
    it = iter(seq)
    result = tuple(itertools.islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

all_arrows = [ go.layout.Annotation(dict(
    ax= z[0],
    ay= len(steps) - i,
    xref="x", yref="y",
    text="",
    showarrow=True,
    axref = "x", ayref='y',
    x= z[1],
    y= len(steps) - 1 - i,
    arrowhead = 3,
    arrowwidth=1.5,
    arrowcolor='rgb(255,51,0)',)
) for i, z in enumerate(window(steps, 2)) ]

fig = px.line(x=X, y=Y1, range_y=[-20, 20], range_x=[-rho_val*1.5, 1.5*rho_val])
# fig = px.line(x=X, y=[Y3, Y1, Y2], range_x=[-100,100]) #, range_y=[-100, 100], range_x=[-100, 100])
fig.add_vline(0)
fig.add_hline(0)
fig.add_vrect(x0=-rho_val, x1=-invrho, fillcolor="green", opacity=0.4, line_width=0)
fig.add_vrect(x0=invrho, x1=rho_val, fillcolor="green", opacity=0.4, line_width=0)
fig.update_layout(xaxis_title='', yaxis_title='', annotations = all_arrows)



fig.show()

root of [-2, -5, 7, -4, 1] near 0.1500000: -0.2756822 (steps: 6)
root of [-7.25470938  8.17871946 -4.27568     1.        ] near 0.1500000: 1.9999969 (steps: 6)
root of [ 3.62735946 -2.27568     1.        ] near 0.1500000: 0.8397033 (steps: 50)
root of [-1.43598  1.     ] near 0.1500000: 1.4359800 (steps: 2)
All roots: [-0.27568, 2.0, 0.8397, 1.43598].
Numpy roots: [ 2.       +0.j          1.1378411+1.52731225j  1.1378411-1.52731225j
 -0.2756822+0.j        ]


$$\begin{align}s(z)=z^4-4z^3+7z^2-5z-2\end{align}$$

$$\begin{align}\rho=8.0\end{align}$$

$$\begin{align}z_0\in[0.125,8.0]\end{align}$$

root of [-2, -5, 7, -4, 1] near -6.0000000: -0.2756822 (steps: 10)
